# Software Combining and Counting Network

Zhuopeng Zhang

4/13/2021

Here, we will introduce two basic data structure - tree and combinatorial network, and use them to implement two distributed coordination patterns - combining and counting. We will see how to make some inherently sequential work paralleled.

## Shared counting<sup>[1]<sup/>

Assume we have a pool which is a collection of items that provides `put()` and `get()` methods to insert and remove items. 

One way to  implement a pool by making both `put()` and `get()` synchronized. The lock will cause a sequential bottleneck, forcing all method calls to synchronize, and memory contention.

Alternatively, a pool can become a cyclic array, where each array entry contains either an item or null. We route threads through two counters. Threads calling `put()` increment one counter to choose an array index. Similarly, threads calling `get()` increment another counter.

So we use two bottleneck, two counters, instead of the one bottleneck, the lock.

Next, we will explore how to build highly parallel shared counters to avoid bottlenecks.

### Software combining

One way to build this parallel counter use a pattern called *software combining*. In this example, we introduce the **Combining tree**:

- It is a binary tree. The counter's value is stored at the root.

- Each thread is assigned a leaf and at most 2 threads share 1 leaf.

- A thread starts at its leaf, and works up the tree to the root.



The possible status of a node when a thread go through the tree:

- IDLE: This node is not in use.
- FIRST: One active thread has visited this node, and will return to check whether another passive thread has left a value with which to combine.
- SECOND: A second thread has visited this node and stored a value in the node's value field to be combined with the active thread's value, but the combined operation is not yet complete.
- RESULT: Both threads' operations have been combined and completed, and the second thread's result has been stored in the node's result field.
- ROOT: This value is a special case to indicate that the node is the root, and must be treated specially.

Notes:

When a thread traverse up the tree, it can continue to go up if it is the first thread arriving at the node. We call it active thread. But when a thread reaches a node that has been visited. It is blocked here and passively wait until another thread return from the root. Then it can continue. One thread can be active or passive at different level.

The concurrent traversal of a width 8 combining tree by five threads.<sup>[1]</sup>

    
<img src="images/combining-tree-execution.png" width=700 style="margin: auto;"/>

- Precombining phase: All threads start from leaves. Then thread A which is the fastest stops at the root, while B stops in the middle-level node where it arrived after A, and C stops at the leaf where it arrived after B.


<img src="images/combining-tree-ab.png" width=1400 style="margin: auto;"/>
 

- Combining phase: Thread A returns back to the start leaf and revisits the nodes it visited in the precombining phase, combining its value with values left by other threads. 

- Operation phase: Thread A gets the prior value of the root, and deposits 3 to the result of root. Thread B deposits the combined value as the second value when it arrives at the middle-level node.

<img src="images/combining-tree-cd.png" width=1400 style="margin: auto;"/>

Notes:

- Combining phase: Thread A set the leaf's first value to 1 it carries. Then thread A stops at the middle-level where the precombining phase ended and waits for B. After B release the lock, A locks the node. The node status is SECOND, so A combined the first and second value written by A and B respectively and moves to the root with the combined value 3.


- Distribution phase: Thread A propagates the result down the tree. Thread A firstly reaches the middle-level node which is in SECOND status, and updates the results to be sum of the *prior* value brought from higher up the tree, and the first value. Then A is back to the leaf which status is FIRST. It only reset the node to its initial state.


<img src="images/combining-tree-e.png" width=700 style="margin: auto;"/>

<pre style="font-size: 16px;">
monitor Node {
    enum CStatus {IDLE, FIRST, SECOND, RESULT, ROOT}
    var locked: boolean
    var firstValue, secondValue: integer
    var result: integer
    var cStatus: CStatus
    var parent: Node
    
    {CI: firstValue ≥ 0 ∧ secondeValue ≥ 0 ∧ result ≥ 0}    
    initialization(myParent: Node)
        parent, cStatus, locked, firstValue, secondValue, result := myParent, IDLE, false, 0, 0, 0
      
    procedure precombine() -> (a: bool)
        {cStatus ≠ SECOND ∧ cStatus ≠ RESULT}
        while locked = true do wait()
        if cStatus = IDLE then cStatus := FIRST; a := true
        if cStatus = FIRST then locked, cStatus := true, SECOND; a := false
        if cStatus = ROOT then cStatus := FIRST; a := false
        {cStatus = FIRST ∨ cStstus = SECOND}
    
    procedure combine(combined: integer) -> (a: integer)
        {cStatus = FIRST ∨ cStatus = SECOND}
        while locked = true do wait
        locked, firstValue := true, combined
        if cStatus = FIRST then a := firstValue
        if cStatus = SECOND then a := firstValue + secondValue
        
    procedure op(combined: integer) -> (a : integer)
        {cStatus = ROOT ∨ cStatus = SECOND}
        if cStatus = ROOT then 
            prior, result := result, result + combined
            a := prior
        if cStatus = SECOND then 
            secondValue, locked := combined, false
            signal()
            while cStatus ≠ RESULT do wait()
            locked := false
            signal()
            cStatus := IDLE
            a := result
              
    procedure distribute(prior: integer)
        {cStatus = FIRST ∨ cStatus = SECOND}
        if cStatus = FIRST then cStatus, locked := IDLE, false
        if cStatus = SECOND then result, cStatus := prior + firstValue, RESULT
        signal()       
}
</pre>

<pre style="font-size: 18px;">
class CombiningTree {
    
    {CI: leaf.length×2 = nodes.length+1 ∧  
            (∀ i ∈ 0..leaf.length-1 • leaf(i) = nodes(nodes.length-i-1)) }
    initialization(width: integer) {
        var nodes : array 0..2×width-1 of Node
        var leaf : array 0..width of Node
        
        nodes(0) := Node()
        i := 0
        while i < nodes.length do
            nodes(i), i := Node(nodes((i-1)/2)), i+1
            
        i := 1
        while i < leaf.length do
            leaf(i), i := nodes(nodes.length-i-1), i+1
    }
    
    method getAndIncrement(）-> (prior: integer)
        var stack : array 0..top of Node
        top: integer = 0
        myLeaf := leaf(ThreadID % leaf.length)
        node := myLeaf
        //precombining phase
        while node.precombine() = true do node := node.parent //find the root or locked node
        stop := node
        //combining phase
        combined: integer =1
        node := myLeaf
        while node ≠ stop do 
            combined, stack(top), node, top := 
                    node.combine(combined), node, node.parent, top+1
        //operation phase
        prior := stop.op(combined)
        //distribute phase
        while stack.length ≠ 0 do
            node, top :=stack(top), top-1
            node.distribute(prior)
}

</pre>

Assume we have p threads, the depth of Combining Tree is at least $\log p$. So each increment takes $O(\log p)$.

Time comparison of Java<sup>[2]</sup>, Go, C#


<table>
<tr style="background-color: white;">
    <td>
    <figure style="display: inline-block; text-align: center;">
    <img src="images/tc-combining-diff-ths.png" width=700 /><br/>
    <figcaption style="text-align: center; font-size:15px">Comparison of different thread numbers, each thread counts 1000.</figcaption>
    </figure>
    </td>
    <td>
    <figure style="display: inline-block; text-align: center;">
    <img src="images/tc-combining-diff-countings.png" width=700 /><br/>
    <figcaption style="text-align: center; font-size:15px">Comparison of different counting numbers with 12 threads.</figcaption>
    </figure>
    </td>
</tr> 
</table>



## Next problem

Now we have multiple above counters. How to distribute the threads among the counters so that there are no duplications or omissions.

<figure  style="display: inline-block;" >
  <img src="images/counting-network.png" width=1200 /><br/>
  <figcaption style="text-align: center; font-size:15px">Counting network<sup>[1]</sup></figcaption>
</figure>

## Counting networks

### Balancer

- A balancer is a switch with two states: up and down. If the state is up, the next token exits on the top wire; otherwise it exits on the bottom wire.


<figure  style="display: inline-block;" >
  <img src="images/balancer.png" width=1200 style="margin: auto;"/>
  <figcaption style="text-align: center; font-size:15px;">Balancer<sup>[1]</sup></figcaption>
</figure>


- A balancing network is ***quiescent*** if every token that arrived on an input wire has emerged on an output wire: $\sum x_i = \sum y_i $



## Counting networks

### Balancing network

- Step property: In general, if $ n = \sum x_i $, then when the network is quiescent, $ y_i = \lceil (n-i)/w\rceil$


- A balancing network that satisfies the *step property* is called a **counting network**


<figure style="display:inline-block;">
  <img src="images/counting_network.png" width=1000 /><br/>
  <figcaption style="text-align: center; font-size: 15px;"> Sequential Bitonic Counting network<sup>[1]</sup></figcaption>
</figure>


### Bitonic counting network


- a **Merger[k] network** merge the even subsequence x0, x2, ..., xk-2 of x with the odd subsequence x1,x3,..xk-1 of x

<figure  style="display: inline-block;" >
  <img src="images/Merger.png" width=700 style="margin: auto;"/>
  <figcaption style="text-align: center; font-size:15px;">Merger<sup>[1]</sup></figcaption>
</figure>


- **Bitonic[2k] network** passes the output from two Bitonic[k] networks into a Merger[2k]

<figure  style="display: inline-block;" >
  <img src="images/bitonic-counting-network.png" width=700 style="margin: auto;"/>
  <figcaption style="text-align: center; font-size:15px;">Bitonic network<sup>[1]</sup></figcaption>
</figure>

Notes:

For merger network,

- if the input wire is in the first *width/2*, the token is sent to *half[0]* if i is even and to *half[1]* if i is odd.


- if the input wire is in the second *width/2*, the token is sent to *half[0]* if i is odd and to *half[1]* if i is even.


- finally, a token on wire i is fed to the balancer at layer[i]


For bitonic network,

- if the input wire is in the first *width/2*, then the token is sent through half[0] (upper Bitonic[k), otherwise through half[1] (lower Bitonic[k])


- if a token emerges from the half[0] (upper Bitonic[k]) on wire i, then it is sent to the Merger[2k] from input wire *i*. Otherwise, it goes into the Merger[2k] from input wire *i+width/2* if it emerges from the half[1] (lower Bitonici[k])

<pre style="font-size: 22px;">
monitor Balancer {
    toggle : boolean = true
    
    procedure traverse() -> (a: integer)
        if toggle = true then
            a := 0
        else
            a := 1
        toggle:=!toggle
}

</pre>

<pre style="font-size: 22px;">
class Merger {

    var half : Array 0..h of Merger
    var layer : Array 0..b of Balancer
    var width : integer
    
    {CI: b = width/2 }
    initialization (myWidth: integer)
        width := myWidth
        b := width/2
        i := 0
        while i < width/2 do layer(i), i := Balancer(), i+1
        if width>2 then
            h := 2
            half(0), half(1) := Merger(width/2), Merger(width/2)
    
    { 0 <= input < width }
    method traverse(input : integer) -> (a: integer)
        output := 0
        if width ≤ 2 then 
            a := layer(0).traverser()
        if input < width/2 then 
            output := half(input%2).traverse(input/2)
        else 
            output := half(1-input%2).traverse(input/2)
        a := (2*output) + layer(output).traverse()
}

</pre>


  <pre style="font-size: 22px;">
class Bitonic {
    half : Array 0..h of Bitonic
    merger : Merger
    width : integer
    
    initialization (myWidth : integer)
        width := myWidth
        merger := Merger(width)
        if width > 2 then
            h := 2
            half(0), half(1) := Bitonic(width/2), Bitonic(width/2)
    
    { 0 <= input < width ∧ 0 ≤ subnet ≤ 1 }
    method traverse(input : integer) -> (a : integer)
        output:=0
        subnet:=input/(width/2)
        if width>2 then
            output:=half(subnet).traverse(input/2)
            
        if input ≥ width/2 then 
            a := merger.traverse(width/2 + output)
        else
            a := merger.traverse(output)
}
 </pre>


Bitonic[2k] is a counting network with depth $O(\log^2 (2k))$.<sup>[1]</sup>

Timing comparison of Java, Go, C#:

<table>
<tr style="background-color: white;">
    <td>
    <figure style="display: inline-block; text-align: center;">
    <img src="images/tc-counting-diff-ths.png" width=700 /><br/>
    <figcaption style="text-align: center; font-size:15px">Comparison of different widths, the number of tokens is 10000.</figcaption>
    </figure>
    </td>
    <td>
    <figure style="display: inline-block; text-align: center;">
    <img src="images/tc-counting-diff-tokens.png" width=700 /><br/>
    <figcaption style="text-align: center; font-size:15px">Comparison of counting different number of tokens, the width is 8.</figcaption>
    </figure>
    </td>
</tr>
</table>

## Conclusion:

- The idea behind the combining tree: if two threads read a node at approximately the same time, they combine their increments by adding them together. One thread propagates their combined increments up the tree, while the other waits for the prior thread to complete their combined work.


- The idea behind the counting network: build a balancing network to count the number of tokens that have traversed the network.

## Question:

1. If we want to change the binary combining tree to a trinary tree allowing 3 threads sharing one node, which fields of the node do we need to change? Describe your design.



## Reference: 
1. Herlihy, M., Shavit, N., Luchangco, V., & Spear, M. (2020). The art of multiprocessor programming. Newnes.
2. Subhajit Sahu. (2020, November 13). javaf/combining-tree. Github. https://github.com/javaf/combining-tree/blob/master/Main.java